# Install Dependencies

In [1]:
!pip install openai-whisper ffmpeg transformers torch TTS moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 31.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 85.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 90.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 79.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 49.9 MB/s eta 0:00:00
  Preparin

# Import Required Libraries

In [2]:
import os
import torch
import whisper
import ffmpeg
from transformers import MarianMTModel, MarianTokenizer
from TTS.api import TTS
from moviepy.editor import VideoFileClip, AudioFileClip
from IPython.display import Audio, display, FileLink

  if event.key is 'enter':



# Upload and Extract Audio from Video

In [3]:
def extract_audio(video_path, output_audio_path="audio.wav"):
    """Extracts audio from a video file."""
    os.system(f"ffmpeg -i {video_path} -q:a 0 -map a {output_audio_path} -y")
    return output_audio_path

# Transcribe Audio using Whisper

In [4]:
def transcribe_audio(audio_path):
    """Transcribes speech from an audio file using OpenAI Whisper."""
    model = whisper.load_model("medium")  # Change to 'small', 'medium', or 'large' based on accuracy needs
    result = model.transcribe(audio_path)
    return result["text"]

# Translation Transcription

In [5]:
def translate_text(text, src_lang="en", tgt_lang="es"):
    """Translates text using MarianMT."""
    model_name = f"Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}"
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    translated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return translated_text[0]

# Voice Cloning and Synthesis

In [6]:
def synthesize_speech(text, output_audio_path="dubbed_audio.wav", language="en", speaker="male-en-2"):
    """Synthesizes speech while maintaining speaker characteristics with a male voice."""
    tts = TTS("tts_models/multilingual/multi-dataset/your_tts", gpu=True)

    # Get available speakers
    available_speakers = tts.speakers if hasattr(tts, "speakers") else []

    # Check if the desired male speaker exists
    if speaker not in available_speakers:
        print(f"⚠️ Speaker '{speaker}' not available! Choosing a valid male voice.")
        male_speakers = [s for s in available_speakers if "male" in s.lower()]  # Filter male speakers

        if male_speakers:
            speaker = male_speakers[0]  # Select the first male speaker
        else:
            print("⚠️ No male speakers available! Using default speaker.")
            speaker = available_speakers[0]  # Use the first available speaker

    print(f"✅ Using speaker: {speaker}, Language: {language}")

    # Generate the translated speech with the selected male speaker
    tts.tts_to_file(text, speaker=speaker, language=language, speed=1.2, file_path=output_audio_path)

    return output_audio_path

# Merge Dubbed Audio with Video

In [7]:
def merge_audio_video(video_path, audio_path, output_video_path="dubbed_video.mp4"):
    """Merges the dubbed audio back into the video while preserving the original resolution and aspect ratio."""
    os.system(f"ffmpeg -i {video_path} -i {audio_path} -c:v libx264 -crf 18 -preset slow -c:a aac -b:a 192k -map 0:v -map 1:a -y {output_video_path}")
    return output_video_path

# Run Pipeline

In [8]:
def process_video(video_path, src_lang="en", tgt_lang="es"):
    audio_path = extract_audio(video_path)
    text = transcribe_audio(audio_path)
    translated_text = translate_text(text, src_lang, tgt_lang)
    dubbed_audio = synthesize_speech(translated_text)
    final_video = merge_audio_video(video_path, dubbed_audio)
    return final_video

# Upload and Test

In [9]:
from google.colab import files
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

Saving DubTest.mp4 to DubTest.mp4


# Process Video

In [10]:
output_video = process_video(video_path)

100%|█████████████████████████████████████| 1.42G/1.42G [00:22<00:00, 67.2MiB/s]
  checkpoint = torch.load(fp, map_location=device)

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")



 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--your_tts


 98%|█████████▊| 416M/425M [00:04<00:00, 102MiB/s]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

 > Model's license - CC BY-NC-ND 4.0
 > Check https://creativecommons.org/licenses/by-nc-nd/4.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | 

# Download Final Dubbed Video

In [11]:
display(FileLink(output_video))

/content/dubbed_video.mp4